In [16]:
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.ml.feature import PCA
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType
import os
import numpy as np
import io
from PIL import Image

In [17]:
if SparkSession.builder.getOrCreate().sparkContext:
    SparkSession.builder.getOrCreate().sparkContext.stop()

spark = (SparkSession
             .builder
             .appName('test2-spark')
             .master('local')
             .config("spark.sql.parquet.writeLegacyFormat", 'true')
             .getOrCreate()
)

In [18]:
PATH = os.getcwd()
PATH_Data = PATH+'/data/Test1'
PATH_Result = PATH+'/data/results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

PATH:        /Users/gaeldelescluse/Documents/OpenClassRooms/2.Projets/Projet11/ai-cloud-computing-spark/train
PATH_Data:   /Users/gaeldelescluse/Documents/OpenClassRooms/2.Projets/Projet11/ai-cloud-computing-spark/train/data/Test1
PATH_Result: /Users/gaeldelescluse/Documents/OpenClassRooms/2.Projets/Projet11/ai-cloud-computing-spark/train/data/results


In [19]:
sc = spark.sparkContext
spark

In [20]:
df = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

In [21]:
df.show()

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|file:/Users/gaeld...|2021-09-12 19:25:42|  5656|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5627|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5613|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5606|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5606|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5602|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5599|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5597|[FF D8 FF E0 00 1...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5594|[FF D8 FF E0 00 1...|
|file:

In [22]:
from pyspark.sql.types import ArrayType, IntegerType

def process_image(content):
    img = Image.open(io.BytesIO(content)).convert('L')
    return np.array(img).flatten().tolist()

# Déclaration de la fonction UDF
process_image_udf = udf(process_image, ArrayType(IntegerType()))

# Ajout d'une colonne 'features' en appliquant l'UDF
df_final = df.withColumn("features", process_image_udf(df.content))

# Affichage du DataFrame final
df_final.show()

+--------------------+-------------------+------+--------------------+--------------------+
|                path|   modificationTime|length|             content|            features|
+--------------------+-------------------+------+--------------------+--------------------+
|file:/Users/gaeld...|2021-09-12 19:25:42|  5656|[FF D8 FF E0 00 1...|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5627|[FF D8 FF E0 00 1...|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5613|[FF D8 FF E0 00 1...|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5611|[FF D8 FF E0 00 1...|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5606|[FF D8 FF E0 00 1...|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5606|[FF D8 FF E0 00 1...|[255, 255, 255, 2...|
|file:/Users/gaeld...|2021-09-12 19:25:42|  5602|[FF D8 FF E0 00 1...|[255, 255,

In [23]:
pca = PCA(k=50, inputCol="features", outputCol="pca_features")
model = pca.fit(df_final)
result = model.transform(df_final)

IllegalArgumentException: requirement failed: Column features must be of type class org.apache.spark.ml.linalg.VectorUDT:struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually class org.apache.spark.sql.types.ArrayType:array<int>.

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import BinaryType
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import PCA
from PIL import Image
import numpy as np
import io

# Initialisation de SparkSession
spark = SparkSession.builder.appName("ImageProcessing").getOrCreate()

# Lecture des fichiers binaires
df = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

# Fonction pour traiter l'image
def process_image(content):
    img = Image.open(io.BytesIO(content)).convert('L')
    return Vectors.dense(np.array(img).flatten().tolist())

# Définir l'UDF
process_image_udf = udf(process_image, VectorUDT())

# Ajouter une colonne 'features' avec les données d'image transformées
df_final = df.withColumn("features", process_image_udf(df.content))

# Appliquer PCA
pca = PCA(k=50, inputCol="features", outputCol="pca_features")
model = pca.fit(df_final)
result = model.transform(df_final)

result.show()

24/05/28 10:18:18 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/05/28 10:19:07 ERROR Executor: Exception in task 0.0 in stage 8.0 (TID 41) 4]
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newArray(Native Method)
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2098)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1675)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2496)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2390)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2228)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1687)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInputStream.java:489)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInput

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 53147)
Traceback (most recent call last):
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/site-packages/pyspark/a

ConnectionRefusedError: [Errno 61] Connection refused